Twitter Customer Dataset

In [1]:
cd /content/drive/MyDrive/ENDProgram/Session9/TwitterCustomerSupport

/content/drive/MyDrive/ENDProgram/Session9/TwitterCustomerSupport


In [2]:
!ls

sample.csv  twcs  tweets.tsv  twitter.zip


In [3]:
# !unzip twitter.zip

In [4]:
import pandas as pd
import torchtext
import torch
import time
import random
import math

from torch import nn, optim

In [5]:
df = pd.read_csv('twcs/twcs.csv')
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [6]:
first_inbound = df[pd.isnull(df.in_response_to_tweet_id) & df.inbound]

In [7]:
inbound_and_replies = pd.merge(first_inbound, df, left_on='tweet_id', right_on='in_response_to_tweet_id')

In [8]:
inbound_and_replies = inbound_and_replies[inbound_and_replies.inbound_y ^ True]

In [9]:
inbound_and_replies = inbound_and_replies.replace('\r', '', regex=True)

In [10]:
tweets = inbound_and_replies[['text_x', 'text_y']]
tweets.columns = ['tweet', 'response']
tweets.head()

,tweet,response
0,@sprintcare is the worst customer service,@115712 Can you please send us a private messa...
1,@sprintcare is the worst customer service,@115712 I would love the chance to review the ...
2,@sprintcare is the worst customer service,@115712 Hello! We never like our customers to ...
3,@115714 y’all lie about your “great” connectio...,@115713 H there! We'd definitely like to work ...
4,"@115714 whenever I contact customer support, t...",@115715 Please send me a private message so th...


In [11]:
tweets.shape

(794299, 2)

In [12]:
tweets.head(100_000).to_csv('tweets.tsv', index=False, sep='\t')

In [13]:
TWEET = torchtext.data.Field(
    tokenize='spacy',
    lower=True,
    init_token='<sos>', 
    eos_token='<eos>'
)

fields = [('tweet', TWEET), ('response', TWEET)]

In [14]:
!pwd

/content/drive/My Drive/ENDProgram/Session9/TwitterCustomerSupport


In [15]:
start = time.time()
dataset = torchtext.data.TabularDataset(
    path='tweets.tsv',
    format='TSV',
    fields=fields,
    skip_header=True
)
end = time.time()
print(f'Duration: {end - start}')

Duration: 33.73662042617798


In [16]:
(train, valid) = dataset.split(split_ratio=[0.85, 0.15])
print(len(train), len(valid))

85000 15000


In [17]:
MAX_VOCAB_SIZE = 10_000
TWEET.build_vocab(train, max_size=MAX_VOCAB_SIZE)
print(f'Size of vocab: {len(TWEET.vocab)}')

Size of vocab: 10004


In [18]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [19]:
BATCH_SIZE = 128

train_iterator, valid_iterator = torchtext.data.BucketIterator.splits(
    (train, valid),
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.tweet),
    device = device
)

In [20]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [21]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [22]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [23]:

INPUT_DIM = len(TWEET.vocab)
OUTPUT_DIM = len(TWEET.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [24]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(128, 256)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(384, 256)
    (fc_out): Linear(in_features=640, out_features=10004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,763,092 trainable parameters


In [26]:
optimizer = optim.Adam(model.parameters())

In [27]:
TRG_PAD_IDX = TWEET.vocab.stoi[TWEET.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [28]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.tweet
        trg = batch.response
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [29]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.tweet
            trg = batch.response

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [30]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [31]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 4m 53s
	Train Loss: 5.052 | Train PPL: 156.274
	 Val. Loss: 5.831 |  Val. PPL: 340.758
Epoch: 02 | Time: 4m 54s
	Train Loss: 4.372 | Train PPL:  79.181
	 Val. Loss: 5.721 |  Val. PPL: 305.155
Epoch: 03 | Time: 4m 52s
	Train Loss: 3.953 | Train PPL:  52.105
	 Val. Loss: 5.646 |  Val. PPL: 283.216
Epoch: 04 | Time: 4m 54s
	Train Loss: 3.746 | Train PPL:  42.347
	 Val. Loss: 5.614 |  Val. PPL: 274.213
Epoch: 05 | Time: 4m 54s
	Train Loss: 3.611 | Train PPL:  36.985
	 Val. Loss: 5.689 |  Val. PPL: 295.676
Epoch: 06 | Time: 4m 53s
	Train Loss: 3.518 | Train PPL:  33.723
	 Val. Loss: 5.688 |  Val. PPL: 295.165
Epoch: 07 | Time: 4m 53s
	Train Loss: 3.462 | Train PPL:  31.894
	 Val. Loss: 5.666 |  Val. PPL: 288.760
Epoch: 08 | Time: 4m 53s
	Train Loss: 3.396 | Train PPL:  29.835
	 Val. Loss: 5.718 |  Val. PPL: 304.227
Epoch: 09 | Time: 4m 53s
	Train Loss: 3.342 | Train PPL:  28.285
	 Val. Loss: 5.839 |  Val. PPL: 343.577
Epoch: 10 | Time: 4m 53s
	Train Loss: 3.329 | Train PPL